In [1]:
import numpy as np
import random
import matplotlib.pyplot as plt
import os
import random
from sklearn.model_selection import train_test_split
import cv2
import torch
import torchvision
import torchvision.transforms as transforms
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

### Coding a simple model

In [ ]:
class MLP:
    def __init__(self, neurons_in_layer, learning_rate):
        self.neurons_in_layer = neurons_in_layer   # 1 input layer, 2 hidden, 1 output layer so a 1x4 array
        self.learning_rate = learning_rate
        
        self.layers = len(neurons_in_layer) # Excluding final layer

        # Initializing weights & biases
        self.weights = []
        self.biases = []
        

        for i in range(self.layers - 1):
            # In the matrices rows correspond to neurons in the current layer and columns to neurons in the next layer
            self.weights.append(np.random.uniform(-1,1,size=[neurons_in_layer[i],neurons_in_layer[i+1]]))
            self.biases.append(np.zeros((1, neurons_in_layer[i+1])))

        print(len(self.weights[0]))
        print(len(self.weights[1]))
        print(len(self.weights[2]))    
    def sigmoid(self, x):
        return 1./(1.+np.exp(-x))
    
    def sigmoid_derivative(self, x):
        sigmoid_x = self.sigmoid(x)
        return sigmoid_x * (1 - sigmoid_x)
    
    def ReLU(self, x):
        return np.maximum(0, x)
    
    def ReLU_derivative(self, x):
        return (x>0)*1
    
    # To have probabilities (all sum up to 1) in the output layer
    def softmax(self, x):
        print("x:", x)
        exp_x = np.exp(x)
        print("exp_x:", exp_x)
        prob = exp_x / exp_x.sum(axis=0, keepdims=True)
        print("prob:", prob)
        return prob
    
    def feed_forward(self, batch):
        self.layer_output = [batch]

        # Passing through each layer except the last one
        for i in range(self.layers - 1):
            output = self.layer_output[-1].dot(self.weights[i]) + self.biases[i]   # weight * x + bias
            self.layer_output.append(self.ReLU(output))

        # Using softmax in the output layer
        self.layer_output[-1] = self.softmax(self.layer_output[-1])
        
        print(self.layer_output[-1])
        return self.layer_output[-1]
        #final_x = np.dot(self.layer_output[-1], self.weights[-1]) + self.biases[-1]
        #self.x_values.append(final_x)
        #self.layer_output.append(self.softmax(final_x))

    def backpropagation(self, y, X_size):
        output_error = self.layer_output[-1] - y  # cross- entropy loss
        delta = output_error * self.ReLU_derivative(self.layer_output[-1])

        # Backpropagation through each layer backwards (duh)
        for i in range(1, self.layers - 1):
            self.weights[-i] -= self.learning_rate * (self.layer_output[-i-1].T.dot(delta))
            self.biases[-i] -= self.learning_rate * np.mean(delta, axis=0, keepdims=True)
            delta = self.ReLU_derivative(self.layer_output[-i-1]) * (delta.dot(self.weights[-i].T))
            # delta = deltas[-1].dot(self.weights[i+1].T) * self.ReLU_derivative(self.layer_output[i+1])
            # deltas.append(delta)

        # # Reversing deltas to match layer order
        # deltas.reverse()

        # # Updating weights and biases
        # for i in range(self.layers - 1):
        #     self.weights[i] -= self.learning_rate * self.layer_output[i].T.dot(deltas[i])
        #     self.biases[i] -= self.learning_rate * np.sum(deltas[i], axis=0, keepdims=True)

    def train(self, X, y, epochs = 1000):
        for epoch in range(epochs):
            output = self.feed_forward(X)
            self.backpropagation(y, len(X))
            #print(output)
            if epoch % 1 == 0:
                loss = -np.sum(y * np.log(output + 1e-9)) / X.shape[0]
                print(f'Epoch {epoch}, Loss: {loss}')

    def predict(self, X):
        output = self.feed_forward(X)
        print("Output:", output)
        print("Label:", np.argmax(output, axis=1))
        return np.argmax(output, axis=1)

    

In [2]:
class MLP_new:
    def __init__(self, input, hidden_layer1, hidden_layer2, output, learning_rate, sample_size):
        self.input = input
        self.hidden_layer1 = hidden_layer1
        self.hidden_layer2 = hidden_layer2
        self.output = output
        self.learning_rate = learning_rate
        
        self.sample_size = sample_size
        self.total_error = 0

        # Initializing weights and biases
        self.weights1 = np.random.uniform(-1, 1, size=[input, hidden_layer1])
        self.weights2 = np.random.uniform(-1, 1, size=[hidden_layer1, hidden_layer2])
        self.weights3 = np.random.uniform(-1, 1, size=[hidden_layer2, output])

        self.biases1 = np.zeros((1, hidden_layer1))
        self.biases2 = np.zeros((1, hidden_layer2))
        self.biases3 = np.zeros((1, output))

        self.final_output = []
        self.loss = []


    # Defining useful functions
    def sigmoid(self, x):
        x_shifted = x - np.max(x, axis=1, keepdims=True)
        return 1/(1+np.exp(-x_shifted * 1.0))
        
    def sigmoid_prime(self, x):
        s = self.sigmoid(x)
        return s * (1 - s)
    
    def ReLU(self, x):
        return np.maximum(0, x)
    
    def ReLU_prime(self,x):
        return (x>0)*1
    
    def softmax(self,x):
        x_shifted = x - np.max(x, axis=1, keepdims=True)
        exp = np.exp(x_shifted)
        probabilities = exp / np.sum(exp, axis=1, keepdims=True)
        return probabilities
    
    def cross_entropy_loss(self, y):
        loss = 0
        # Loop through each sample
        for sample in range(y.shape[0]):
            # Get the predicted probabilities for the current sample
            probabilities = self.final_output[sample]
            
            # Ensure numerical stability by adding a small constant (epsilon) to avoid log(0)
            epsilon = 1e-9
            probabilities = np.clip(probabilities, epsilon, 1 - epsilon)
            
            # Compute the loss for the current sample using the correct one-hot encoded label
            loss += -np.sum(y[sample] * np.log(probabilities))
        
        # Return the average loss
        return loss / y.shape[0]

    # Forward
    def train(self, X, y, epochs):
        self.loss = np.zeros([epochs,1])
        for epoch in range(epochs):
            self.final_output = np.zeros([self.sample_size, self.output])
            for sample, inputs in enumerate(X):
                inputs = inputs.reshape(-1,1).T
                # Forward pass
                output1 = self.sigmoid(inputs.dot(self.weights1) + self.biases1)
                output2 = self.sigmoid(output1.dot(self.weights2) + self.biases2)
                output3 = self.softmax(output2.dot(self.weights3) + self.biases3)

                self.final_output[sample] = output3

                # Backward pass
                error = output3 - y[sample]
                self.total_error += error
                
                delta_out = error
                self.weights3 -= self.learning_rate * (output2.T.dot(delta_out))
                self.biases3 -= self.learning_rate * delta_out
        
                delta2 = self.sigmoid_prime(output2) * (delta_out.dot(self.weights3.T))
                self.weights2 -= self.learning_rate * (output1.T.dot(delta2))
                self.biases2 -= self.learning_rate * delta2

                delta1 = self.sigmoid_prime(output1) * (delta2.dot(self.weights2.T))
                self.weights1 -= self.learning_rate * (inputs.T.dot(delta1))
                self.biases1 -= self.learning_rate * delta1

            self.total_error = self.total_error/len(y)

            self.loss[epoch] = self.cross_entropy_loss(y)
            if epoch % 10 == 0:
                print(f"Epoch: {epoch}, Loss: {self.loss[epoch]}, Final output: {self.final_output}")

    def predict(self, X):
        predictions = []
        self.final_output = np.zeros([self.sample_size, self.output])
        for sample, inputs in enumerate(X):
            # Forward pass
            inputs = inputs.reshape(-1,1).T
                # Forward pass
            output1 = self.sigmoid(inputs.dot(self.weights1) + self.biases1)
            output2 = self.sigmoid(output1.dot(self.weights2) + self.biases2)
            output3 = self.softmax(output2.dot(self.weights3) + self.biases3)

            self.final_output[sample] = output3

            predictions.append(np.argmax(self.final_output[sample]))
        return predictions

### Loading data

In [3]:
data_file = "C:/Users/afrod/Documents/Neural_Networks/MergedDataset"
classes = ["NonDemented", "VeryMildDemented", "MildDemented", "ModerateDemented"]
training_data = []


def create_training_data():
    for dementia_level in classes:
        path = os.path.join(data_file, dementia_level)
        class_num = classes.index(dementia_level)
        for img in os.listdir(path):
            # Convert to grayscale for smaller array dimensions
            img_array = cv2.imread(os.path.join(path,img), cv2.IMREAD_GRAYSCALE)
            final_array = cv2.resize(img_array, (50,47))
            training_data.append([final_array, class_num])

create_training_data()

In [19]:
random.shuffle(training_data)

# Separating features and labels
# Images are also flattened to be used as input in the knn algorithm
X = np.array([features for features, _ in training_data]).reshape(-1, 50*47)
y = np.array([label for _, label in training_data])

# Rescaling
X = X/X.max()

# One-hot encoding
y_onehot = np.zeros((y.size, int(y.max()) + 1))
y_onehot[np.arange(y.size),y.astype(int)] = 1.0

# Splitting data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y_onehot, test_size=0.4, random_state=42)
print(X_train.shape[0])

24230


### Training time

In [30]:
mlp = MLP_new(2350, 100, 100, 4, 0.001, len(X_train))

In [31]:
mlp.train(X_train, y_train, 50)

Epoch: 0, Loss: [1.3886602], Final output: [[0.48388275 0.12694301 0.28514547 0.10402877]
 [0.36181644 0.12767838 0.41670766 0.09379751]
 [0.38558649 0.14663354 0.37741558 0.09036438]
 ...
 [0.23591263 0.26018052 0.29187884 0.212028  ]
 [0.26696231 0.28430223 0.26309477 0.18564069]
 [0.38328613 0.25992176 0.20999112 0.14680098]]


C:\Users\afrod\AppData\Local\Temp\ipykernel_19844\964597294.py:28: RuntimeWarning: overflow encountered in exp
  return 1/(1+np.exp(-x_shifted * 1.0))


Epoch: 10, Loss: [1.36150616], Final output: [[0.29151286 0.26891269 0.26177083 0.17780361]
 [0.29499867 0.26748832 0.26036553 0.17714748]
 [0.29339741 0.26616841 0.26378175 0.17665243]
 ...
 [0.29114728 0.27305349 0.25634224 0.17945699]
 [0.28960492 0.27169546 0.25973982 0.1789598 ]
 [0.29308501 0.27025127 0.25836218 0.17830153]]
Epoch: 20, Loss: [1.36149841], Final output: [[0.29151708 0.26891155 0.26175025 0.17782112]
 [0.29499459 0.26749038 0.26034875 0.17716629]
 [0.29339758 0.26617356 0.26375699 0.17667187]
 ...
 [0.29115168 0.27304257 0.25633379 0.17947195]
 [0.28961331 0.27168783 0.25972348 0.17897537]
 [0.29308512 0.27024693 0.25834952 0.17831842]]
Epoch: 30, Loss: [1.36149063], Final output: [[0.29152136 0.26891033 0.26172959 0.17783871]
 [0.29499054 0.26749237 0.2603319  0.1771852 ]
 [0.29339778 0.26617866 0.26373215 0.17669141]
 ...
 [0.29115615 0.27303154 0.25632532 0.17948698]
 [0.28962177 0.2716801  0.2597071  0.17899103]
 [0.29308528 0.2702425  0.25833682 0.1783354 ]]
E

In [32]:
# Evaluate accuracy on test set
predictions = mlp.predict(X_test)
y_test_labels = np.argmax(y_test, axis=1)
accuracy = np.mean(predictions == y_test_labels)
print("\nMLP with layers [2350, 100, 100, 4], lr = 0.001, epochs = 50:")
print(f'Test Accuracy: {accuracy * 100:.2f}%')
print("Accuracy:", accuracy_score(y_test_labels, predictions))
print("Classification Report:\n", classification_report(y_test_labels, predictions))
print("Confusion Matrix:\n", confusion_matrix(y_test_labels, predictions))

C:\Users\afrod\AppData\Local\Temp\ipykernel_19844\964597294.py:28: RuntimeWarning: overflow encountered in exp
  return 1/(1+np.exp(-x_shifted * 1.0))



MLP with layers [2350, 100, 100, 4], lr = 0.001, epochs = 50:
Test Accuracy: 31.68%
Accuracy: 0.31682555404234247
Classification Report:
               precision    recall  f1-score   support

           0       0.32      1.00      0.48      5118
           1       0.00      0.00      0.00      4467
           2       0.00      0.00      0.00      3937
           3       0.00      0.00      0.00      2632

    accuracy                           0.32     16154
   macro avg       0.08      0.25      0.12     16154
weighted avg       0.10      0.32      0.15     16154

Confusion Matrix:
 [[5118    0    0    0]
 [4467    0    0    0]
 [3937    0    0    0]
 [2632    0    0    0]]


c:\Users\afrod\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\afrod\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\afrod\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, mo

In [26]:
mlp.loss

array([[1.39312199],
       [1.39214072],
       [1.39088064],
       [1.38959126],
       [1.38830926],
       [1.38704071],
       [1.38579065],
       [1.38456388],
       [1.38336492],
       [1.382198  ],
       [1.38106678],
       [1.37997431],
       [1.37892297],
       [1.37791449],
       [1.37694998],
       [1.37602983],
       [1.37515392],
       [1.37432167],
       [1.373532  ],
       [1.37083346],
       [1.36671748],
       [1.36617092],
       [1.36552665],
       [1.36492847],
       [1.36442928],
       [1.36387694],
       [1.36323595],
       [1.36285675],
       [1.36249574],
       [1.36206912],
       [1.36160188],
       [1.36118851],
       [1.36084999],
       [1.36046136],
       [1.36023572],
       [1.35992535],
       [1.35973884],
       [1.35957772],
       [1.35962157],
       [1.35946076],
       [1.359187  ],
       [1.35892768],
       [1.35873198],
       [1.35879359],
       [1.3588937 ],
       [1.35897807],
       [1.35849076],
       [1.358

In [27]:
loss_mat = np.matrix(mlp.loss)

with open('n_100_0.01_50.txt','wb') as f:
    for line in loss_mat:
        np.savetxt(f, line, fmt='%.5f')

In [33]:
# Evaluate accuracy on test set
predictions = mlp.predict(X_train)
y_test_labels = np.argmax(y_train, axis=1)
accuracy = np.mean(predictions == y_test_labels)
print("\nMLP with layers [2350, 100, 100, 4], lr = 0.001, epochs = 50:")
print(f'Train Accuracy: {accuracy * 100:.2f}%')
print("Accuracy:", accuracy_score(y_test_labels, predictions))
print("Classification Report:\n", classification_report(y_test_labels, predictions))
print("Confusion Matrix:\n", confusion_matrix(y_test_labels, predictions))

C:\Users\afrod\AppData\Local\Temp\ipykernel_19844\964597294.py:28: RuntimeWarning: overflow encountered in exp
  return 1/(1+np.exp(-x_shifted * 1.0))



MLP with layers [2350, 100, 100, 4], lr = 0.001, epochs = 50:
Train Accuracy: 31.70%
Accuracy: 0.31704498555509697
Classification Report:
               precision    recall  f1-score   support

           0       0.32      1.00      0.48      7682
           1       0.00      0.00      0.00      6733
           2       0.00      0.00      0.00      5919
           3       0.00      0.00      0.00      3896

    accuracy                           0.32     24230
   macro avg       0.08      0.25      0.12     24230
weighted avg       0.10      0.32      0.15     24230

Confusion Matrix:
 [[7682    0    0    0]
 [6733    0    0    0]
 [5919    0    0    0]
 [3896    0    0    0]]


c:\Users\afrod\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\afrod\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\afrod\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, mo

In [3]:
# Cell 2: Load CIFAR-10 dataset
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=1000, shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=1000, shuffle=False, num_workers=2)

def extract_features_labels(dataloader):
    features, labels = [], []
    for images, lbls in dataloader:
        # Flatten images and append to list
        features.extend(images.view(images.size(0), -1).numpy())
        labels.extend(lbls.numpy())
    return np.array(features), np.array(labels)

train_features, train_labels = extract_features_labels(trainloader)
test_features, test_labels = extract_features_labels(testloader)

y_onehot_train = np.zeros((train_labels.size, int(train_labels.max()) + 1))
y_onehot_train[np.arange(train_labels.size),train_labels.astype(int)] = 1.0

y_onehot_test = np.zeros((test_labels.size, int(test_labels.max()) + 1))
y_onehot_test[np.arange(test_labels.size),test_labels.astype(int)] = 1.0


NameError: name 'transforms' is not defined